In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import datetime
now = datetime.datetime.now()

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
year = 2019 # Year we're currently testing.
html = requests.get('https://en.wikipedia.org/wiki/List_of_Billboard_Hot_100_top-ten_singles_in_' + str(year),cookies={ 'sessionid': '123..'},headers=headers).text
soup = BeautifulSoup(html, 'lxml')
print(soup.prettify()[0:1000])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of Billboard Hot 100 top-ten singles in 2019 - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XgESlwpAMFUAAAvZyyMAAACC","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_Billboard_Hot_100_top-ten_singles_in_2019","wgTitle":"List of Billboard Hot 100 top-ten singles in 2019","wgCurRevisionId":932136634,"wgRevisionId":932136634,"wgArticleId":59536348,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":

In [3]:
my_table = soup.find('table',{'class':'wikitable sortable'})
type(my_table)

bs4.element.Tag

In [4]:
tableheaderincluded = True
swiftcount = 0 # Keeps track of where repeat information from cells with rowspans greater than 1 should be inserted.
drakecontingency = 0 # Used to keep track of how many times repeat information should be inserted.

# The following code mines the first table found on the Wikipedia page and extracts all table elements other than the annotations column.

In [5]:
Titles = []
try:
    tds = my_table.findAll('td')
except AttributeError:
    my_table = soup.find('table',{'class':'wikitable'})
    tableheaderincluded = False
    tds = my_table.findAll('td')

for td in tds:
        try:
            if td.attrs['rowspan']:
                taylorswift = td.get_text()[:-1]
                swiftcount = 7
                drakecontingency = int(td.attrs['rowspan']) - 1
        except KeyError:
            pass
        if td.get_text()[0] != '[' and td.get_text()[0] and td.get_text()[0].strip(): # Drop annotations column.
            if swiftcount == 1:
                drakecontingency = drakecontingency-1;
                print(taylorswift + ' appended!')
                Titles.append(taylorswift)
                if drakecontingency == 0:
                    taylorswift = ''
                    swiftcount = 0
                else:
                    swiftcount = 6;
            if swiftcount > 0:
                swiftcount = swiftcount-1;
            Titles.append(td.get_text()[:-1]) # Drop the newline character present at the end of every single entry.

October 20 appended!
January 5 appended!
January 5 appended!
April 13 appended!
April 27 appended!
May 18 appended!
June 29 appended!
September 21 appended!
October 19 appended!
November 9 appended!
November 9 appended!


In [6]:
print(Titles)

['December 30(2017)', '"All I Want For Christmas Is You"[U]', 'Mariah Carey', '1', 'December 21', '9*', 'October 20', '"Shallow"[C]', 'Lady Gaga and Bradley Cooper', '1', 'March 9', '7', 'October 20', '"Happier"(#6)', 'Marshmello and Bastille', '2', 'February 16', '27', 'November 3', '"Sunflower"(#2)[J][Q] ◁', 'Post Malone and Swae Lee', '1', 'January 19', '33', 'November 10', '"Without Me"(#3)', 'Halsey', '1', 'January 12', '29', 'November 24', '"High Hopes"', 'Panic! at the Disco', '4', 'January 26', '15', 'January 5', '"Jingle Bell Rock"[W]', 'Bobby Helms', '8', 'January 5', '2*', 'January 5', '"Rockin\' Around the Christmas Tree"[U]', 'Brenda Lee', '2', 'December 28', '4*', 'January 5', '"Holly Jolly Christmas"[V]', 'Burl Ives', '6', 'December 28', '3*', 'January 19', '"Eastside"[B]', 'Benny Blanco, Halsey and Khalid', '9', 'January 19', '2', 'January 26', '"Wow"(#5)', 'Post Malone', '2', 'April 6', '24', 'February 2', '"7 Rings"(#7) ◁', 'Ariana Grande', '1', 'February 2', '16', 'F

In [7]:
nptitles = np.asarray(Titles).copy()

In [10]:
len(nptitles)

324

In [11]:
numsongs = int(np.prod(nptitles.shape)/6) # Find out how many songs are in this list.
reshaped = nptitles.reshape(numsongs,6) # Reshape the array based on the number of songs and the number of columns.

In [17]:
songdata = pd.DataFrame(reshaped)
songdata.columns = ['Top ten entry date','Title','Artist','Peak','Peak date','Number of weeks in top ten']
songdata['Year'] = str(year)
songdata['Title'] = [i[1:i.rfind('\"')] for i in songdata['Title']] # Drops quotation marks and annotations in Title.
for j in songdata['Top ten entry date']:
    if '[' in j:
        j = j[:j.rfind('[')-1]

if year == now.year:
    cleanweeks = []
    for j in songdata['Number of weeks in top ten']: # This code searches the number of weeks in top ten column for asterisks and eliminates them. This is only a 
        try:                                         # problem in the current year.
            cleanweeks.append(int(j[:j.rindex('*')]))
        except ValueError:
            cleanweeks.append(int(j))

    songdata['Number of weeks in top ten'] = cleanweeks

In [18]:
songdata

,Top ten entry date,Title,Artist,Peak,Peak date,Number of weeks in top ten,Year
0,December 30(2017),All I Want For Christmas Is You,Mariah Carey,1,December 21,9,2019
1,October 20,Shallow,Lady Gaga and Bradley Cooper,1,March 9,7,2019
2,October 20,Happier,Marshmello and Bastille,2,February 16,27,2019
3,November 3,Sunflower,Post Malone and Swae Lee,1,January 19,33,2019
4,November 10,Without Me,Halsey,1,January 12,29,2019
5,November 24,High Hopes,Panic! at the Disco,4,January 26,15,2019
6,January 5,Jingle Bell Rock,Bobby Helms,8,January 5,2,2019
7,January 5,Rockin' Around the Christmas Tree,Brenda Lee,2,December 28,4,2019
8,January 5,Holly Jolly Christmas,Burl Ives,6,December 28,3,2019
9,January 19,Eastside,"Benny Blanco, Halsey and Khalid",9,January 19,2,2019


In [19]:
songdata.to_csv(path_or_buf='data/' + str(year) + 'top10songs.csv',index=False) # Export our sample data table.